In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Load the dataset
complaints_df = pd.read_excel('/content/drive/MyDrive/Team 3 - Complaints.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from gensim.models import Word2Vec


# Load the Excel file
df = pd.read_excel("/content/drive/MyDrive/Team 3 - Complaints.xlsx")


# Function to preprocess a single narrative
def preprocess_narrative(narrative):
    if pd.isna(narrative):
        return ''  # Return an empty string for NaN values


    # Tokenize the narrative
    tokens = word_tokenize(str(narrative).lower())  # Convert to lowercase and tokenize


    # Remove strings with numbers, punctuation, or 'xxxx'
    tokens = [token for token in tokens if re.match(r'^[a-zA-Z]+$', token)]


    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]


    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]


    return tokens


# Apply preprocessing to all narratives in the DataFrame
df['tokens'] = df['narrative'].apply(preprocess_narrative)

# Train Word2Vec model
model = Word2Vec(df['tokens'], vector_size=100, window=5, min_count=1, workers=4)


# Example of accessing word vectors
print(model.wv['customer'])


import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


# Load the dataset
complaints_df = pd.read_excel("/content/drive/MyDrive/dataset/Copy of Team 3 - Complaints.xlsx")


# Preprocess the narratives
def preprocess_narrative(narrative):
    if pd.isna(narrative):
        return ''
    # Remove non-alphabetic characters and convert to lowercase
    clean_narrative = re.sub(r'[^a-zA-Z\s]', '', str(narrative).lower())
    return clean_narrative


complaints_df['processed_narrative'] = complaints_df['narrative'].apply(preprocess_narrative)


# Tokenize narratives
tokenized_narratives = complaints_df['processed_narrative'].apply(lambda x: x.split())


# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_narratives, vector_size=100, window=5, min_count=1, workers=4)


# Generate word embeddings
def generate_word_embeddings(narrative, model):
    word_embeddings = []
    for word in narrative.split():
        if word in model.wv:
            word_embeddings.append(model.wv[word])
    return np.mean(word_embeddings, axis=0) if word_embeddings else np.zeros(model.vector_size)


complaints_df['word_embeddings'] = complaints_df['processed_narrative'].apply(lambda x: generate_word_embeddings(x, word2vec_model))

X = np.vstack(complaints_df['word_embeddings'])
y = complaints_df['product']


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)


# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
